# RNN test v2

## ref: https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/recurrent_network.ipynb


Notes to remind myself:

treating the 3D data as a series of 2D images over time, collapsing the third spatial dimension and treating it as a sequence of 2D images.


In [1]:
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import numpy as np
import glob
import nibabel as nib
import os
import matplotlib.pyplot as plt
import scipy.ndimage
import random
from tensorflow.keras.layers import Dropout, Dense, Reshape, Flatten, Conv3D, Conv3DTranspose, LeakyReLU, Input, Embedding, multiply, Concatenate
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout, BatchNormalization
from functools import partial
from tensorflow.keras import models, layers

c:\Users\josep\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# CNN layers for spatial feature extraction, 
# Bidirectional LSTM for better sequence learning, 
# Dropout for regularization.

from tensorflow.keras import layers, models
from tensorflow.keras.layers import Bidirectional, AdditiveAttention, LayerNormalization

def build_rnn_model():
    time_steps = 146  # Number of images per sequence
    features_per_image = 91 * 91  # Each image is flattened into a vector of this size
    
    # Input layer
    inputs = Input(shape=(time_steps, features_per_image))
    
    # Bidirectional LSTM layers
    x = Bidirectional(layers.LSTM(256, return_sequences=True))(inputs)
    x = LayerNormalization()(x)
    x = Bidirectional(layers.LSTM(256, return_sequences=True))(x)
    
    # Attention mechanism
    attention = AdditiveAttention()([x, x])  # Use LSTM output as both query and key/value
    
    # Combine attention output with LSTM output
    x = layers.Concatenate()([x, attention])
    
    # Another Bidirectional LSTM layer
    x = Bidirectional(layers.LSTM(128))(x)
    
    # Dropout for regularization
    x = layers.Dropout(0.5)(x)
    
    # Dense output layer
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    # Build the model
    model = Model(inputs, outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [3]:
rnn_model = build_rnn_model()
rnn_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 146, 8281)]  0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 146, 512)     17485824    ['input_1[0][0]']                
                                                                                                  
 layer_normalization (LayerNorm  (None, 146, 512)    1024        ['bidirectional[0][0]']          
 alization)                                                                                       
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 146, 512)    1574912     ['layer_normalization[0][0]']

In [4]:
full_schizophrenia_ids = [
    'A00009280', 'A00028806', 'A00023132', 'A00014804', 'A00016859', 'A00021598', 'A00001181', 'A00023158',
    'A00024568', 'A00028405', 'A00001251', 'A00000456', 'A00015648', 'A00002405', 'A00027391', 'A00016720',
    'A00018434', 'A00016197', 'A00027119', 'A00006754', 'A00009656', 'A00038441', 'A00012767', 'A00034273',
    'A00028404', 'A00035485', 'A00024684', 'A00018979', 'A00027537', 'A00004507', 'A00001452', 'A00023246',
    'A00027410', 'A00014719', 'A00024510', 'A00000368', 'A00019293', 'A00014830', 'A00015201', 'A00018403',
    'A00037854', 'A00024198', 'A00001243', 'A00014590', 'A00002337', 'A00024953', 'A00037224', 'A00027616',
    'A00001856', 'A00037619', 'A00024228', 'A00038624', 'A00037034', 'A00037649', 'A00022500', 'A00013216',
    'A00020787', 'A00028410', 'A00002480', 'A00028303', 'A00020602', 'A00024959', 'A00018598', 'A00014636',
    'A00019349', 'A00017147', 'A00023590', 'A00023750', 'A00031597', 'A00015518', 'A00018317', 'A00016723',
    'A00021591', 'A00023243', 'A00017943', 'A00023366', 'A00014607', 'A00020414', 'A00035003', 'A00028805',
    'A00029486', 'A00000541', 'A00028408', 'A00000909', 'A00031186', 'A00000838' ]

# schizohrenia_id that satisfy t>90, 59 in total
met_requirement_schizophrenia_ids = [
    'A00000368', 'A00000456', 'A00000541', 'A00000838', 'A00001251', 'A00001452', 'A00004507',
    'A00006754', 'A00009280', 'A00012767', 'A00013216', 'A00014607', 'A00014719', 'A00014804',
    'A00014830', 'A00015201', 'A00015648', 'A00016197', 'A00016720', 'A00016723', 'A00017147',
    'A00018317', 'A00018403', 'A00018434', 'A00018979', 'A00019293', 'A00020414', 'A00020602', 
    'A00020787', 'A00021591', 'A00021598', 'A00023158', 'A00023246', 'A00023590', 'A00023750', 
    'A00024198', 'A00024228', 'A00024568', 'A00024684', 'A00024953', 'A00024959', 'A00027410', 
    'A00027537', 'A00028303', 'A00028404', 'A00028408', 'A00028805', 'A00028806', 'A00031186', 
    'A00031597', 'A00034273', 'A00035003', 'A00035485', 'A00037034', 'A00037224', 'A00037619', 
    'A00037649', 'A00038441', 'A00038624']

full_control_ids = [
    'A00007409', 'A00013140', 'A00021145', 'A00036049', 'A00022810', 'A00002198', 'A00020895', 'A00004667',
    'A00015826', 'A00023120', 'A00022837', 'A00010684', 'A00009946', 'A00037318', 'A00033214', 'A00022490',
    'A00023848', 'A00029452', 'A00037564', 'A00036555', 'A00023095', 'A00022729', 'A00024955', 'A00024160',
    'A00011725', 'A00027487', 'A00024446', 'A00014898', 'A00015759', 'A00028409', 'A00017294', 'A00014522',
    'A00012995', 'A00031764', 'A00025969', 'A00033147', 'A00018553', 'A00023143', 'A00036916', 'A00028052',
    'A00023337', 'A00023730', 'A00020805', 'A00020984', 'A00000300', 'A00010150', 'A00024932', 'A00035537',
    'A00022509', 'A00028406', 'A00004087', 'A00035751', 'A00023800', 'A00027787', 'A00022687', 'A00023866',
    'A00021085', 'A00022619', 'A00036897', 'A00019888', 'A00021058', 'A00022835', 'A00037495', 'A00026945',
    'A00018716', 'A00026907', 'A00023330', 'A00016199', 'A00037238', 'A00023131', 'A00014120', 'A00021072',
    'A00037665', 'A00022400', 'A00003150', 'A00024372', 'A00021081', 'A00022592', 'A00022653', 'A00013816',
    'A00014839', 'A00031478', 'A00014225', 'A00013363', 'A00037007', 'A00020968', 'A00024301', 'A00024820',
    'A00035469', 'A00029226', 'A00022915', 'A00022773', 'A00024663', 'A00036844', 'A00009207', 'A00024535',
    'A00022727', 'A00011265', 'A00024546'
]

 # 82 controls that met requirement
met_requirement_control_ids = [
    'A00000300', 'A00002198', 'A00003150', 'A00004087', 'A00007409', 'A00010684', 'A00011265', 'A00011725',
    'A00012995', 'A00013140', 'A00013816', 'A00014839', 'A00014898', 'A00015759', 'A00015826', 'A00018553',
    'A00018716', 'A00019888', 'A00020805', 'A00020895', 'A00020968', 'A00020984', 'A00021058', 'A00021072',
    'A00021081', 'A00021085', 'A00022400', 'A00022490', 'A00022509', 'A00022592', 'A00022619', 'A00022653',
    'A00022687', 'A00022727', 'A00022729', 'A00022773', 'A00022810', 'A00022835', 'A00022837', 'A00022915',
    'A00023095', 'A00023120', 'A00023131', 'A00023143', 'A00023330', 'A00023337', 'A00023730', 'A00023800',
    'A00023848', 'A00023866', 'A00024160', 'A00024301', 'A00024372', 'A00024446', 'A00024535', 'A00024546', 
    'A00024663', 'A00024820', 'A00024932', 'A00024955', 'A00025969', 'A00026945', 'A00027487', 'A00027787', 
    'A00028052', 'A00028406', 'A00028409', 'A00029226', 'A00029452', 'A00031478', 'A00031764', 'A00033214', 
    'A00035751', 'A00036049', 'A00036555', 'A00036844', 'A00037007', 'A00037238', 'A00037318', 'A00037495', 
    'A00037564', 'A00037665'
]

In [5]:
# GAN Training Data Selection
gan_train_ids_schiz = random.sample(met_requirement_schizophrenia_ids, 4)
gan_test_ids_schiz = [id for id in met_requirement_schizophrenia_ids if id not in gan_train_ids_schiz]

gan_train_ids_control = random.sample(met_requirement_control_ids, 4)
gan_test_ids_control = [id for id in met_requirement_control_ids if id not in gan_train_ids_control]
gan_test_ids_control = random.sample(gan_test_ids_control,4)

''' data training for classifier '''
''' just use the same train set as GAN above '''

# Classifier Test Data Selection
classifier_test_ids = gan_test_ids_schiz + gan_test_ids_control

''' File loading '''
# Specify the directory and file pattern
directory_path = '4D/'
file_pattern = 'A*_????_func_FL_FD_RPI_DSP_MCF_SS_SM_Nui_CS_InStandard.nii.gz'

# Construct the full path pattern
path_pattern = f'{directory_path}/{file_pattern}'

# Use glob to find all matching files
matching_files = glob.glob(path_pattern)

''' File loading for GAN Training and classifer '''
''' But this time we have 2 separate GANs, 1 train on schizoprenia and 1 train on control'''

#classifier_image_data = []
#classifier_labels = []  # 1 for schizophrenia, 0 for non-schizophrenia
gan_image_data_schiz = []
gan_image_data_control = []

for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in gan_train_ids_schiz:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()
        

        if t1_data.shape[3] < 90:
            continue

        t1_data_collapsed = np.sum(t1_data, axis=1)
        print('shape of image after collapsed: ', t1_data_collapsed.shape)
        gan_image_data_schiz.append(t1_data_collapsed)

    if file_id in gan_train_ids_control:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        if t1_data.shape[3] < 90:
            continue

        t1_data_collapsed = np.sum(t1_data, axis=1)
        gan_image_data_control.append(t1_data_collapsed)


print(f"Total GAN control loaded: {len(gan_image_data_control)}")
print(f"Total GAN schiz loaded: {len(gan_image_data_schiz)}")



'''Determine the maximum 3rd-dimension size '''
max_z_size_schiz = max(img.shape[2] for img in gan_image_data_schiz)
max_z_size_control = max(img.shape[2] for img in gan_image_data_control)

max_z_size = max(max_z_size_schiz,max_z_size_control)

''' normalization '''
image_data_normalized_schiz = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in gan_image_data_schiz]
image_data_normalized_control = [(img - np.min(img)) / (np.max(img) - np.min(img)) * 2 - 1 for img in gan_image_data_control]


''' padding of images data '''
# Pad each image to have a consistent z-dimension size
padded_data_schiz = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size - img.shape[2])), mode='constant') for img in image_data_normalized_schiz]
padded_data_control = [np.pad(img, ((0, 0), (0, 0), (0, max_z_size - img.shape[2])), mode='constant') for img in image_data_normalized_control]


# Now convert the padded data list to a numpy array
padded_data_array_schiz = np.array(padded_data_schiz)
padded_data_array_control = np.array(padded_data_control)


''' loading the data for WGAN training '''
train_images_schiz = padded_data_array_schiz
train_images_control = padded_data_array_control

# Define batch size
batch_size = 10


train_dataset_schiz = tf.data.Dataset.from_tensor_slices((train_images_schiz)).shuffle(len(train_images_schiz)).batch(batch_size)
train_dataset_control = tf.data.Dataset.from_tensor_slices((train_images_control)).shuffle(len(train_images_control)).batch(batch_size)

shape of image after collapsed:  (91, 91, 141)
shape of image after collapsed:  (91, 91, 123)
shape of image after collapsed:  (91, 91, 145)
shape of image after collapsed:  (91, 91, 124)
Total GAN control loaded: 4
Total GAN schiz loaded: 4


In [6]:
# Flatten each 2D image slice and prepare it for RNN input
train_images_schiz = np.reshape(padded_data_array_schiz, (len(padded_data_array_schiz), 146, 91*91))
train_images_control = np.reshape(padded_data_array_control, (len(padded_data_array_control), 146, 91*91))

# Create labels
labels_schiz = np.ones(len(train_images_schiz))
labels_control = np.zeros(len(train_images_control))

# Combine the datasets
train_images = np.concatenate((train_images_schiz, train_images_control), axis=0)
train_labels = np.concatenate((labels_schiz, labels_control), axis=0)

# Shuffle the dataset
indices = np.arange(train_images.shape[0])
np.random.shuffle(indices)
train_images = train_images[indices]
train_labels = train_labels[indices]

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)


In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')

# Define number of epochs
epochs = 15

# Training loop
for epoch in range(epochs):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    
    for images, labels in train_dataset:
        with tf.GradientTape() as tape:
            predictions = rnn_model(images, training=True)
            loss = tf.keras.losses.binary_crossentropy(labels, tf.squeeze(predictions, axis=1))
        
        gradients = tape.gradient(loss, rnn_model.trainable_variables)
        rnn_model.optimizer.apply_gradients(zip(gradients, rnn_model.trainable_variables))
        
        # Update the metrics
        train_loss.update_state(loss)
        train_accuracy.update_state(labels, tf.squeeze(predictions, axis=1))
    
    # Print the loss and accuracy for the current epoch
    print(f"Epoch {epoch + 1}, Loss: {train_loss.result().numpy()}, Training Accuracy: {train_accuracy.result().numpy()}")


Epoch 1, Loss: 0.6389578580856323, Training Accuracy: 0.625
Epoch 2, Loss: 0.6340192556381226, Training Accuracy: 0.5
Epoch 3, Loss: 0.8936426639556885, Training Accuracy: 0.625
Epoch 4, Loss: 0.7858017683029175, Training Accuracy: 0.25
Epoch 5, Loss: 0.6199522018432617, Training Accuracy: 0.5
Epoch 6, Loss: 0.526227593421936, Training Accuracy: 0.625
Epoch 7, Loss: 0.5648474097251892, Training Accuracy: 0.5
Epoch 8, Loss: 0.5173529386520386, Training Accuracy: 0.75
Epoch 9, Loss: 0.4888719916343689, Training Accuracy: 0.625
Epoch 10, Loss: 0.33151063323020935, Training Accuracy: 1.0
Epoch 11, Loss: 0.5690021514892578, Training Accuracy: 0.75
Epoch 12, Loss: 0.5714426040649414, Training Accuracy: 0.75
Epoch 13, Loss: 0.5867455005645752, Training Accuracy: 0.625
Epoch 14, Loss: 0.4800736904144287, Training Accuracy: 0.75
Epoch 15, Loss: 0.45470190048217773, Training Accuracy: 0.75


In [8]:
def resize_image(image, new_shape):
    factors = (
        new_shape[0] / image.shape[0],
        new_shape[1] / image.shape[1],
        new_shape[2] / image.shape[2]
    )
    return scipy.ndimage.zoom(image, factors, order=1)  # order=1 is bilinear interpolation


test_image_data = []
test_labels = []

test_ids = classifier_test_ids  # List of IDs to filter test data

for file_path in matching_files:
    filename = os.path.basename(file_path)
    file_id = filename.split('_')[0]
    
    if file_id in test_ids:
        t1_img = nib.load(file_path)
        t1_data = t1_img.get_fdata()

        if t1_data.shape[3] < 90:
            continue
        
        t1_data_collapsed = np.sum(t1_data, axis=1)  # Summing over one axis

        # Resize the collapsed data
        processed_image = resize_image(t1_data_collapsed, (91, 91, 146))

        # Normalize the processed image
        processed_image_normalized = (processed_image - np.min(processed_image)) / (np.max(processed_image) - np.min(processed_image)) * 2 - 1

        # Reshape for RNN input: Flatten each 91x91 image to a single vector and treat each as a timestep
        processed_image_flattened = np.reshape(processed_image_normalized, (146, 91*91))

        test_image_data.append(processed_image_flattened)
        label = 1 if file_id in met_requirement_schizophrenia_ids else 0
        test_labels.append(label)

# Convert to numpy arrays for easier handling in TensorFlow
test_images_array = np.array(test_image_data)
test_labels_array = np.array(test_labels)

# Create a TensorFlow dataset from the numpy arrays
test_dataset = tf.data.Dataset.from_tensor_slices((test_images_array, test_labels_array)).batch(batch_size)


In [9]:
# Evaluate the model
loss, accuracy = rnn_model.evaluate(test_dataset)
print(f"Test loss: {loss}, Test accuracy: {accuracy}")

6/6 [==============================] - 3s 92ms/step - loss: 2.1895 - accuracy: 0.2203
Test loss: 2.1895229816436768, Test accuracy: 0.22033898532390594


In [10]:
# Predict the probabilities
predictions = rnn_model.predict(test_dataset)
# Convert probabilities to class labels
predicted_labels = (predictions > 0.5).astype(int)

# Assuming test_labels_array contains your actual labels
actual_labels = test_labels_array

# Now you might want to compare these predicted_labels with the actual labels (test_labels)
# to compute the confusion matrix, classification report, etc.
from sklearn.metrics import classification_report, confusion_matrix

print(confusion_matrix(actual_labels, predicted_labels.flatten()))
print(classification_report(actual_labels, predicted_labels.flatten()))

6/6 [==============================] - 2s 86ms/step
[[ 4  0]
 [46  9]]
              precision    recall  f1-score   support

           0       0.08      1.00      0.15         4
           1       1.00      0.16      0.28        55

    accuracy                           0.22        59
   macro avg       0.54      0.58      0.21        59
weighted avg       0.94      0.22      0.27        59



In [11]:
for actual, predicted in zip(actual_labels, predicted_labels):
    print(f'Actual: {actual}, Predicted: {predicted}')

Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 0, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 0, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [0]
Actual: 1, Predicted: [1]
Actual: 1, Predicted: [0]
Actual: 1, P